# Mask R-CNN - Evaluation on Prostate Dataset


This notebook shows how to use trained Mask R-CNN on prostate dataset for evaluation. As for large pathology image, we crop each image to several patches. This notebook is designed to get the detection reulst for single patch and evaluate them use mIOU. You'd need a GPU, though, because the network backbone is a Resnet101, which would be slow to detect on a CPU.

The code of the Prostate dataset can be found in prostate.py.

## Import Module

In [ ]:
# import module from system lib
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scipy.io

In [ ]:
# import module from maskrcnn repo
from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate
import pydensecrf.densecrf as dcrf

%matplotlib inline

# Specify GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

## Configurations

In [ ]:
# Specify the dir that store the prostate dataset
dataset_dir = os.path.join('/data/wenyuan/Path_R_CNN', "Data_Pre_Processing/cedars-224")
# We do 5-fold validation, specify which fold to be exclude for the current run
held_out_set = 4
# Featch the mean_pixel based on the training data (data exclude the held_out_set)
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
# Configuration
class EvaluationConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5
    DETECTION_NMS_THRESHOLD = 1     
    MEAN_PIXEL = np.array(mean_pixel)
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_DIM = 512
    DETECTION_CROP = [128, 384, 128, 384] # [height_crop_start, height_crop_end, width_crop_start, width_crop_end]
    MODE = 16
    USE_TUMORCLASS = True
evaluation_config = EvaluationConfig()
evaluation_config.display()

## Create Model Graph and Loading Weights

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="detection", 
                          config=evaluation_config,
                          model_dir=MODEL_DIR)
# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights
h5_filename = None # Specify the h5 filename here if you want to choose a specific file
epoch = 71

if h5_filename is not None:
    model_path = os.path.join(ROOT_DIR, ".h5 file name here")
elif epoch == -1:    
    model_path = model.find_last()[1]
else:
    try:
        model_path = model.find_specific(epoch = epoch, held_out_set = 0)[1]
    except:
        model_path = model.find_specific(epoch = epoch)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

## Prepare the Dataset

In [ ]:
dataset_val = prostate.ProstateDataset()
_, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)
# val_list = [image for image in val_list if image not in exclude_list]
dataset_val.load_prostate(dataset_dir, val_list, mode = 16)
dataset_val.prepare()

## Run Evaluation

In [ ]:
# Initialize the confusion matrix
C_MATRIX = np.zeros((4, 4))
# Create crop region
hv, wv = utils.create_crop_region(evaluation_config) # meshgrid for crop region
# Process display setting
display_step = 10 # print out process for every display_step images
total_image = len(val_list)
for image_id in dataset_val.image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, evaluation_config,
                                   image_id, use_mini_mask=False)
    # Convert gt-instance mask to gt-sementic mask
    gt_sementic_mask = utils.instance_2_sementic(gt_mask, gt_class_id)
    gt_sementic_mask = gt_sementic_mask['ATmask'][hv, wv] # crop the label
    # Run object detection
    results = model.detect([image], verbose=0)
    # TODO: create a function for evaluation in model
    r = results[0]    
#     if np.argmax(r['tumor_probs']) == 1 and len(r['class_ids']) != 0:
#         det_sementic_mask = r['sementic_mask']
#     else:
#         det_sementic_mask = np.zeros((image.shape[0], image.shape[1]))
    ## This section only enabled wo Diagnosit Network Head (DNH)
    if len(r['class_ids']) != 0:
        det_sementic_mask = r['sementic_mask']
    else:
        det_sementic_mask = np.zeros((image.shape[0], image.shape[1]))
    
    det_sementic_mask = det_sementic_mask[hv, wv] # crop the detection    
    # Compute confusion matrix
    c_matrix = confusion_matrix(np.ravel(gt_sementic_mask), np.ravel(det_sementic_mask))
    # Expand the c_matrix to NUM_CLASSES * NUM_CLASSES
    c_matrix = utils.expand_c_matrix(c_matrix, evaluation_config.NUM_CLASSES, gt_sementic_mask, det_sementic_mask)    
    # Update cofusion matrix
    C_MATRIX = C_MATRIX + c_matrix
    # Display the process
    if ((image_id + 1) % (evaluation_config.MODE * display_step) == 0):
        print('Done evaluating %d / %d!'%((image_id + 1) / evaluation_config.MODE, total_image))

## Evaluation Results

In [ ]:
mIOU, IOU, below_th = utils.compute_mIOU(C_MATRIX, th = 0.5)
print('Confusion Matrix:\n', C_MATRIX)
print(' mIOU:', mIOU, '\n', 
      'IOU for each class:', IOU, '\n',
      'Below_th:', below_th)

In [ ]:
import scipy.io
filename = "Set%d_wo_DNH"%held_out_set
C_Matrix_dict = {filename:C_MATRIX}

In [ ]:
file_path = filename + ".mat"
scipy.io.savemat(file_path, C_Matrix_dict)